In [1]:
import os
import numpy as np
import random
import csv
import librosa
import tensorflow as tf
from keras import Sequential, regularizers
from tensorflow.keras.layers import Conv2D, Conv1D, MaxPooling2D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, Input, LSTM, TimeDistributed
from sklearn.model_selection import train_test_split

In [2]:
SAMPLE_RATE = 8000
DURATION = 5
N_MFCC = 20
N_CLASSES = 2  # количество# классов: картавый или не картавый
MAX_LEN = int(SAMPLE_RATE * DURATION) + 1  # максимальная длина MFCC
BATCH_SIZE = 64

In [10]:
# Функции для аугментации

def add_noise(data, noise_factor=0.005):
    noise = np.random.randn(data.shape[0])
    augmented_data = data + noise_factor * noise
    return augmented_data


def change_speed(data, speed_factor=1.2):
    if speed_factor < 0.5:
        speed_factor = 0.5
    return librosa.effects.time_stretch(data, rate=speed_factor)

# Функции для загрузки данных

def load_audio_files(file_paths):
    data = []
    
    for file_path in file_paths:
        y, sr = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)
        factors = np.random.normal(1, 1, 2)

        
        y = add_noise(y, factors[0]) # Добавление шума и изменение скорости
        y = change_speed(y, abs(factors[1]))

        # Извлечение признаков при помощи MFCC
        mfcc = librosa.feature.mfcc(y=y, sr=sr) # Взятие MFCC
        mfcc = y
        if mfcc.shape[0] < MAX_LEN: # Дополнение до необходимого размера
            pad_width = MAX_LEN - mfcc.shape[1]
            mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:

            mfcc = mfcc[:, :MAX_LEN]
        mfcc = np.expand_dims(mfcc, axis=-1)  # Добавление дополнительного измерения для каналов
        

        # Извлечение признаков при помощи спектрограммы
        # mfcc = librosa.feature.melspectrogram(y=y, sr=sr) # Взятие MFCC
        # if mfcc.shape[0] < MAX_LEN:
        #     pad_width = MAX_LEN - mfcc.shape[1]
        #     mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
        # else:

        #     mfcc = mfcc[:, :MAX_LEN]
        # mfcc = np.expand_dims(mfcc, axis=-1)  # Добавление дополнительного измерения для каналов

        # data.append(mfcc)

        # Наивный подход
        # if len(y) < MAX_LEN:
        #     pad_width = MAX_LEN - len(y)
        #     y = np.pad(y, pad_width=((pad_width)), mode='constant')
        # if len(y) > MAX_LEN:
        #     y = y[:MAX_LEN]
        # data.append(y)
        # y = np.expand_dims(y, axis=-1)
        # print(y.shape)
    return np.array(data, dtype=np.float32)

def load_data(data_dir):

    file_paths = []
    labels = []

    with open(data_dir) as file:
        reader = csv.reader(file, delimiter=',')
        for row in reader:
            file_path = f'train/{row[0]}'
            file_paths += [file_path]
            labels += [[int(row[1]) == 0, int(row[1]) != 0]]
    print(len(file_paths))
    X = load_audio_files(file_paths)
    y = np.array(labels)
    return X, y

In [4]:
# Загружаем данные

data_dir = 'train_gt.csv'
X, y = load_data(data_dir)


8803


MemoryError: Unable to allocate 19.5 MiB for an array with shape (128, 40001) and data type float32

In [10]:
X_train, X_test, y_train, y_test = X[:8400], X[8400:], y[:8400, :2], y[8400:, :2] 

In [11]:
print(X_train[0].shape)

(40001, 1)


In [7]:
# Первая модель - CNN с 3-мя свертками. Используется для MFCC и спектрограммы. Гиперпараметры подбирались максимально низкие при которых модель показывает
# хорошую точность на обучающей выборке, для избежания переобучения. Результаты на тестовой выборке спустя 10 эпох: 
# MFCC - 65%, Спектрограмма - 61% (обучение происходит медленнее из-за большего размера)

model1 = Sequential([
    Input((20, 235, 1), batch_size=BATCH_SIZE),
    Conv2D(16, (2, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(24, (2, 3), activation='relu'), 
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(32, (2, 3), activation='relu'), 
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(N_CLASSES, activation='softmax')
])

In [25]:
# Вторая модель - одномерная CNN для поиска по аудиоряду. Результаты спустя 10 эпох на тестовой выборке - 61%. Обучение медленное из-за большой сложности

model2 = Sequential([
    Input((40001, 1), batch_size=BATCH_SIZE),
    Conv1D(4, (25), activation='relu', strides=2),
    BatchNormalization(),
    MaxPooling1D(5),
    Conv1D(8, 25, activation='relu'), 
    BatchNormalization(),
    MaxPooling1D(5),
    Conv1D(12, 25, activation='relu'), 
    BatchNormalization(),
    MaxPooling1D(5),
    Conv1D(4, 25, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(5),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(N_CLASSES, activation='softmax')
])

In [8]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) # Для обучения всях моделей использовался Adam. 
model1.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [9]:

# Обучение модели
for i in range(40):
    model1.fit(X_train, y_train, epochs=1, batch_size=BATCH_SIZE)
    
    # Оценка модели
    loss, precision, recall = model1.evaluate(X_test, y_test)
    print(f'Test Loss: {loss}')
    print(f'Test Accuracy: {2 * precision * recall / (precision + recall)}') 

NameError: name 'X_train' is not defined

0
0
0
1
0
0
0
1
0
1
1
0
1
1
0
1
0
1
0
1
0
1
1
1
0
0
0
1
1
1
0
1
1
0
1
0
1
1
1
0
0
0
0
0
0
1
1
1
0
0
0
1
0
1
0
1
1
0
1
1
0
0
0
1
0
1
1
1
1
0
1
0
0
0
1
1
1
1
0
1
0
1
0
1
0
1
1
1
1
1
1
0
0
1
0
1
1
1
0
1
1
1
0
1
0
1
0
0
1
1
1
0
0
1
1
1
0
0
1
1
0
1
1
1
0
0
1
1
1
1
1
0
1
1
1
0
0
1
1
0
0
1
1
1
1
1
0
0
0
0
1
1
1
0
0
0
0
1
0
0
0
1
1
1
1
1
1
0
1
1
0
0
0
0
0
1
1
0
1
0
0
0
1
1
0
0
1
0
0
1
0
1
1
1
1
0
1
0
0
1
1
1
1
0
0
0
1
0
1
1
0
0
1
1
1
1
0
0
1
0
0
1
0
1
0
1
1
1
1
0
0
0
1
0
1
1
1
1
0
1
1
1
0
1
0
1
1
1
1
0
1
0
0
1
1
0
0
0
0
1
0
0
0
1
1
0
1
1
1
0
0
0
0
1
0
1
0
0
1
0
0
1
0
1
1
0
0
0
1
1
1
1
1
0
1
0
1
1
0
0
0
0
1
0
1
0
1
1
1
1
0
1
1
1
1
0
0
0
0
0
1
0
1
1
1
1
1
1
1
1
0
0
1
0
1
0
0
1
0
1
1
0
1
1
0
1
0
1
0
0
0
0
0
1
0
0
0
1
1
1
0
1
1
1
0
1
0
1
1
0
0
1
0
0
1
0
0
1
0
1
1
0
0
1
1
1
0
0
0
1
1
0
1
1
0
1
1
1
1
1
0
1
1
0
0
0
0
1
0
0
1
1
1
0
0
1
0
1
1
0
1
1
1
0
1
0
0
1
0
1
1
1
1
0
1
0
0
0
1
1
0
0
0
1
1
1
1
0
1
0
1
1
1
0
0
0
0
1
1
0
1
0
1
0
1
0
1
0
0
1
0
0
1
0
1
0
1
1
0
0
1
1
0
1
0
1
0
0
1
1
1
0
1
1
1
0
0
1
0
1
